---
# **6. Оптимальная стратегия управления портфелем (HJB)**

Динамика капитала портфеля с весами $\mathbf{w}$:

$$\frac{dX_t}{X_t} = \left[r_f + \mathbf{w}^T(\boldsymbol{\mu} - r_f \mathbf{1})\right] dt + \mathbf{w}^T \boldsymbol{\sigma} dW_t$$

### Функция полезности CRRA

$$U(X) = \frac{X^{1-\gamma}}{1-\gamma}$$

где $\gamma$ — коэффициент риск-аверсии.

### Оптимальные веса

Решение HJB уравнения:
$$\mathbf{w}^* = \frac{1}{\gamma} \Sigma^{-1} (\boldsymbol{\mu} - r_f \mathbf{1})$$

**Интерпретация**:
- Меньший $\gamma$ → больший leverage (рискованнее)
- Больший $\gamma$ → консервативнее

### Ожидаемая доходность оптимального портфеля

$$\mu_p^* = r_f + \frac{1}{\gamma} \text{SR}^2$$

где $\text{SR}$ — максимальный Sharpe ratio.

### Волатильность

$$\sigma_p^* = \frac{1}{\gamma} \sqrt{(\boldsymbol{\mu} - r_f \mathbf{1})^T \Sigma^{-1} (\boldsymbol{\mu} - r_f \mathbf{1})}$$

### Sharpe Ratio

$$\text{SR}^* = \sqrt{(\boldsymbol{\mu} - r_f \mathbf{1})^T \Sigma^{-1} (\boldsymbol{\mu} - r_f \mathbf{1})}$$

**Важно**: Sharpe не зависит от $\gamma$!

### Ограничения

- **Запрет коротких продаж**: $w_i \geq 0$
- **Нормировка**: $\sum w_i = 1$ (остаток в risk-free)
- **Плечо**: $\sum |w_i| \leq 1$

---


In [ ]:
rf_manual = 13.94

print(f" Безрисковая ставка (r_f) для HJB-стратегии:\n")
print(f"   КБД Мосбиржи: {rf_manual}% годовых")
print(f"   (Установите актуальное значение на дату начала симуляции)")

rf_hjb = rf_manual / 100

print(f" Используется r_f = {rf_hjb:.4f} ({rf_manual}%) для расчёта избыточных доходностей\n")

class HJBStrategy:

    def __init__(self, mu, cov_matrix, risk_free_rate, gamma, asset_names=None,
                 short_sales_allowed=False, max_leverage=1.0, min_weight=0.0,
                 normalize_weights=True):

        self.mu = np.asarray(mu).flatten()
        self.cov_matrix = np.asarray(cov_matrix)
        self.risk_free_rate = float(risk_free_rate)
        self.gamma = float(gamma)
        self.n_assets = len(self.mu)
        self.asset_names = asset_names if asset_names is not None else \
                          [f"Asset_{i+1}" for i in range(self.n_assets)]

        self.short_sales_allowed = short_sales_allowed
        self.max_leverage = max_leverage
        self.min_weight = min_weight
        self.normalize_weights = normalize_weights

        if self.gamma <= 0:
            raise ValueError(f"γ должен быть > 0, получено {self.gamma}")

        if self.cov_matrix.shape != (self.n_assets, self.n_assets):
            raise ValueError(
                f"Размерность ковариационной матрицы {self.cov_matrix.shape} "
                f"не соответствует количеству активов {self.n_assets}"
            )


        if self.max_leverage <= 0:
            raise ValueError(f"max_leverage должен быть > 0, получено {self.max_leverage}")

        self.excess_mu = self.mu - self.risk_free_rate

        try:
            self.inv_cov_matrix = np.linalg.inv(self.cov_matrix)
        except np.linalg.LinAlgError:
            print("  Матрица вырождена, применяется псевдообратная")
            self.inv_cov_matrix = np.linalg.pinv(self.cov_matrix)

        self._weights_base = np.dot(self.inv_cov_matrix, self.excess_mu)

        print(f" HJB стратегия инициализирована:")
        print(f"   Активов:              {self.n_assets}")
        print(f"   γ (риск-аверсия):     {self.gamma}")
        print(f"   r_f (безрисковая):    {self.risk_free_rate*100:.2f}% (КБД Мосбиржи)")
        print(f"   Короткие продажи:     {'Разрешены' if self.short_sales_allowed else 'Запрещены'}")
        print(f"   Макс. плечо:          {self.max_leverage}")
        print(f"   Мин. вес актива:      {self.min_weight*100:.2f}%")
        print(f"   Нормировка весов:     {'Да (Σw=1)' if self.normalize_weights else 'Нет (γ влияет на leverage)'}")

        print(f"\n Проверка избыточных доходностей (μ - r_f):")
        for i, ticker in enumerate(self.asset_names):
            excess = self.excess_mu[i]
            emoji = "🟢" if excess > 0 else "🔴"
            print(f"   {emoji} {ticker:5s}: {excess*100:>7.2f}%")

        print(f"\n   Средняя избыточная доходность: {np.mean(self.excess_mu)*100:>7.2f}%")


    def get_optimal_weights(self):

        weights_raw = (1.0 / self.gamma) * self._weights_base

        if not self.short_sales_allowed:
            weights_clipped = np.clip(weights_raw, 0, None)
        else:
            weights_clipped = weights_raw.copy()

        sum_weights = np.sum(np.abs(weights_clipped))


        if sum_weights < 1e-10:
            print(f"  Все веса ≤ 0, применяется равномерное распределение")
            return np.ones(self.n_assets) / self.n_assets


        if self.normalize_weights:

            weights_final = weights_clipped / np.sum(weights_clipped)
        else:

            current_leverage = np.sum(weights_clipped)


            if current_leverage > self.max_leverage:
                weights_final = weights_clipped * (self.max_leverage / current_leverage)
            else:
                weights_final = weights_clipped

        if self.min_weight > 0:
            weights_final[weights_final < self.min_weight] = 0.0

            if self.normalize_weights:
                sum_w = np.sum(weights_final)
                if sum_w > 1e-10:
                    weights_final = weights_final / sum_w

        return weights_final



    def update(self, t, state):

        X_t = state['capital']


        if X_t <= 1e-6:
            return np.zeros(self.n_assets)

        weights = self.get_optimal_weights()

        pi_t = weights * X_t


        return pi_t

    def get_portfolio_stats(self):

        weights = self.get_optimal_weights()
        leverage = np.sum(np.abs(weights))

        expected_return = np.dot(weights, self.mu)

        portfolio_variance = np.dot(weights, np.dot(self.cov_matrix, weights))
        volatility = np.sqrt(portfolio_variance)

        sharpe_ratio = (expected_return - self.risk_free_rate) / volatility if volatility > 1e-10 else 0.0

        return {
            'weights': weights,
            'leverage': leverage,
            'expected_return': expected_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio
        }

    def print_weights(self):

        weights = self.get_optimal_weights()
        leverage = np.sum(weights)

        print(f"\n Оптимальные веса портфеля (γ = {self.gamma}, r_f = {self.risk_free_rate*100:.2f}%):\n")

        sorted_indices = np.argsort(weights)[::-1]

        for idx in sorted_indices:
            weight = weights[idx]
            if weight > 0.001:
                bar = "█" * int(weight * 50)
                print(f"   {self.asset_names[idx]:6s}: {weight*100:>6.2f}% {bar}")

        print(f"\n   Σ весов (leverage): {leverage:.4f}")
        print(f"   Активных позиций:   {np.sum(weights > 0.001)}")

print(" Итоговые переменные:\n")
print(f"   rf_hjb               : {rf_hjb:.4f} ({rf_manual}%) — безрисковая ставка (КБД Мосбиржи)")
print(f"   HJB стратегия          : класс оптимальной стратегии управления портфелем")
print(f"   Параметры:")
print(f"     - normalize_weights=True  → веса Σ=1, γ не влияет на leverage")
print(f"     - normalize_weights=False → γ влияет на leverage (Σw ≤ max_leverage)\n")